In [18]:
import sqlite3
from sqlite3 import Error
import requests

In [33]:
inventory_sheet_url = 'https://docs.google.com/spreadsheets/d/1abfWjKouFxD_erzhdnam8tCtFC5G_956/edit#gid=2096778584'
schedule_sheet_url = 'https://docs.google.com/spreadsheets/d/19jmopenGv7VnXOWeBmPpsicmCZycEfOl/edit#gid=2051886986'
pnr_sheet_url = 'https://docs.google.com/spreadsheets/d/1E3Vnx5WA0ntOG42A_oyNI1VCNMn_UY6938mt9Mxz2gs/edit#gid=0'

In [34]:
def get_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return conn

In [35]:
def get_sheet_data(sheet_url):
    """Returns 2d array of sheet data"""
    sheet_id = sheet_url.split('/')[5]
    gid = sheet_url.split('/')[6].split('=')[1]
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=tsv&gid={gid}"
    response = requests.get(url)
    data = response.text
    rows = data.split('\n')
    sheet_data = []
    for row in rows:
        sheet_data.append(row.split('\t'))
    return sheet_data

In [36]:
def parseDate(date):
    """MM/DD/YYYY to YYYY-MM-DD"""
    m, d, y = date.split('/')
    return f"{y}-{m}-{d}"
def parseDateTime(datetime):
    """MM/DD/YYYY HH:MM:SS to YYYY-MM-DD HH:MM:SS"""
    date, time = datetime.split(' ')
    return parseDate(date) + ' ' + time

In [37]:
conn = get_connection('db.sqlite3')
cur = conn.cursor()

Connection to SQLite DB successful


In [24]:
inventory_data = get_sheet_data(inventory_sheet_url)


In [25]:
for row in inventory_data[1:]:
    inventoryID = row[0]
    scheduleID = row[1]
    departureDate = parseDate(row[4])
    arrivalDate = parseDate(row[5])
    totalCapacity = row[8]
    totalInventory = row[9]
    bookedInventory = row[10]
    cur.execute(f"""
        INSERT INTO Inventory VALUES (
            '{inventoryID}',
            '{scheduleID}',
            '{departureDate}',
            '{arrivalDate}',
            {totalCapacity},
            {totalInventory},
            {bookedInventory}
        );
    """)
    conn.commit()


In [26]:
schedule_data = get_sheet_data(schedule_sheet_url)

In [27]:
for row in schedule_data[1:]:
    scheduleID = row[0]
    flightNumber = row[2]
    departureAirport = row[5]
    arrivalAirport = row[6]
    departureTime = row[7]
    arrivalTime = row[8]
    startDate = row[9]
    endDate = row[10]
    status = row[11]
    scheduleDays = ''.join(row[12:19])
    cur.execute(f"""
        INSERT INTO Schedule VALUES (
            '{scheduleID}',
            '{flightNumber}',
            '{departureAirport}',
            '{arrivalAirport}',
            '{departureTime}',
            '{arrivalTime}',
            '{startDate}',
            '{endDate}',
            '{status}',
            '{scheduleDays}'
        );
    """)
    conn.commit()

In [38]:
pnr_data = get_sheet_data(pnr_sheet_url)

In [39]:
for row in pnr_data[1:]:
    passengerID = row[0]
    flightNumber = row[1]
    departureDate = parseDate(row[2])
    arrivalDate = parseDate(row[3])
    departureTime = row[4]
    arrivalTime = row[5]
    paidOrBooked = row[6]
    cabinClass = row[7]
    cur.execute(f"""
        INSERT INTO PNR VALUES (
            '{passengerID}',
            '{flightNumber}',
            '{departureDate}',
            '{arrivalDate}',
            '{departureTime}',
            '{arrivalTime}',
            '{paidOrBooked}',
            '{cabinClass}'
        );
    """)
    conn.commit()